In [ ]:
filename = dbutils.widgets.get('filename')
print(filename)

In [ ]:
storage_access_key = dbutils.secrets.get(scope = 'databricksScope1', key='storage-access-key')

In [ ]:
alreadyMounted = False
for x in dbutils.fs.mounts():
    if x.mountPoint == '/mnt/sales':
        alreadyMounted = True
        break 
    else:
        alreadyMounted = False

if not alreadyMounted: 
    dbutils.fs.mount(
    source = 'wasbs://sales@datalakeazureproject.blob.core.windows.net',
    mount_point= '/mnt/sales',
    extra_configs = {'fs.azure.account.key.datalakeazureproject.blob.core.windows.net':storage_access_key})
    
    alreadyMounted = True
    print("mounting done successfully")
else:
    print("Already mounted")
        


In [ ]:
%fs 

ls /mnt/sales/order_items



path,name,size,modificationTime
dbfs:/mnt/sales/order_items/order_items.txt,order_items.txt,5485772,1715784654000


In [ ]:
orders_df = spark.read.csv("/mnt/sales/landing/{}".format(filename),inferSchema=True,header=True)

In [ ]:
display(orders_df)

order_id,order_date,customer_id,order_status
1111111,2013-07-25T00:00:00Z,11599,CLOSED
2222222,2013-07-25T00:00:00Z,256,PENDING_PAYMENT
3333333,2013-07-25T00:00:00Z,12111,COMPLETE
4444444,2013-07-25T00:00:00Z,8827,CLOSED


In [ ]:
#First condition check - if duplicate order_id move file to discarded else create a table

errorFlg = False

orders_count = orders_df.count()
print(orders_count)

orders_distinct_count = orders_df.select('order_id').distinct().count()
print(orders_distinct_count)

if orders_count != orders_distinct_count:
    errorFlg = True

if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg":"true","errormsg":"order_id is repeated"}')

orders_df.createOrReplaceTempView('orders')


In [ ]:
#applying second validation - to check order status is valid or not by connecting to azure sql db

dbServer = 'ttsqlserver2580'
dbPort = '1433'
dbName = 'ttsqldb'
dbUser = 'nithin'
dbPassword = 'SQL-password'
databricksScope = 'databricksScope1 '

In [ ]:
connectionUrl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer,
dbPort, dbName, dbUser)

dbPassword =  dbutils.secrets.get(scope = databricksScope, key='SQL-password')

connectionProperties = {'password': dbPassword,'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'}

In [ ]:
#to read the table

valid_status_df = spark.read.jdbc(url =connectionUrl,table='dbo.valid_order_status',properties=connectionProperties)

In [ ]:
display(valid_status_df)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELED
PENDING_PAYMENT


In [ ]:
valid_status_df.createOrReplaceTempView('valid_status')

In [ ]:
invalidRowsdf = spark.sql("select * from orders where order_status not in (select * from valid_status)")

In [ ]:
display(invalidRowsdf)

order_id,order_date,customer_id,order_status


In [ ]:
if invalidRowsdf.count() > 0 :
    errorFlg = True

if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg":"true","errormsg":"order_status is not valid"}')
else:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/staging')
    #dbutils.notebook.exit('{"errorFlg":"false","errormsg":"condition: Pass"}')

In [ ]:
orderItemDf = spark.read.csv('/mnt/sales/order_items/order_items.txt',inferSchema=True,header=True)
orderItemDf.createOrReplaceTempView("order_items")

In [ ]:
customer_df = spark.read.jdbc(url =connectionUrl,table='dbo.customers',properties=connectionProperties)
display(customer_df)
customer_df.createOrReplaceTempView("customers")

customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,00725
6,Mary,Smith,XXXXXXXXX,XXXXXXXXX,3151 Sleepy Quail Promenade,Passaic,NJ,07055
7,Melissa,Wilcox,XXXXXXXXX,XXXXXXXXX,9453 High Concession,Caguas,PR,00725
8,Megan,Smith,XXXXXXXXX,XXXXXXXXX,3047 Foggy Forest Plaza,Lawrence,MA,01841
9,Mary,Perez,XXXXXXXXX,XXXXXXXXX,3616 Quaking Street,Caguas,PR,00725
10,Melissa,Smith,XXXXXXXXX,XXXXXXXXX,8598 Harvest Beacon Plaza,Stafford,VA,22554


In [ ]:
orders_df = spark.read.csv("/mnt/sales/staging/{}".format(filename),inferSchema=True,header=True)
orders_df.createOrReplaceTempView("orders")

In [ ]:
result1_df = spark.sql("select customers.customer_id,customers.customer_fname,customers.customer_lname,customers.customer_city,customers.customer_state,customers.customer_zipcode,count(order_id) as num_orders_placed,round(sum(order_item_subtotal),2) as total_amount from customers,orders,order_items where customers.customer_id = orders.customer_id and orders.order_id=order_items.order_item_order_id group by customers.customer_id,customers.customer_fname,customers.customer_lname,customers.customer_city,customers.customer_state,customers.customer_zipcode order by total_amount desc")

In [ ]:
display(result1_df) 

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 166.0 failed 4 times, most recent failure: Lost task 0.3 in stage 166.0 (TID 134) (10.139.64.4 executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/mnt/sales/landing/orders_new.csv. [DEFAULT_FILE_NOT_FOUND] It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved. If disk cache is stale or the underlying files have been removed, you can invalidate disk cache manually by restarting the cluster.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:700)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:638)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:795)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:505)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:500)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:151)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:57)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$3(ShuffleMapTask.scala:92)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.$anonfun$runTask$1(ShuffleMapTask.scala:87)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:58)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:39)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:196)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:181)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:146)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:129)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:146)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:99)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$8(Executor.scala:933)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:102)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:936)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:828)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.

In [ ]:
result1_df.write.jdbc(url =connectionUrl,table='dbo.sales_reporting',properties=connectionProperties,mode = 'overwrite')